In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# BEWARE! Data was latest available in September 2024

Clean version of datasets below results in:
* IP ranges, countries and ASN: total of 82396 unique ASN
* ASN and categories: total of 107111 unique ASN
* ASN, org names and countries: total of 117878 unique ASN


# Initial inspection and data formatting
Detaisls as number of rows and cols, datatypes (if object, will need to further inspect), head of the df.


In [ ]:
route = '/kaggle/input/asn-categories-countries-ips-raw/country_asn.csv'
df_asn_cat_count_ips_raw = pd.read_csv(route, low_memory=False)
route = '/kaggle/input/asn-categories-countries-ips-raw/2024-01_categorized_ases.csv'
df_cat_as_raw = pd.read_csv(route, low_memory=False)
route = '/kaggle/input/asn-categories-countries-ips-raw/20240901.as-org2info.jsonl'
df_as_info_raw = pd.read_json(route, lines=True)


In [ ]:
df_asn_cat_count_ips_raw.info()
df_asn_cat_count_ips_raw.head()

In [ ]:
df_cat_as_raw.info()
df_cat_as_raw.head()

In [ ]:
df_as_info_raw.info()
df_as_info_raw.head()

## IP ranges, countries and ASN

In [ ]:
## code for reading CSV files
df_asn_cat_count_ips_raw.head()
df_asn_cat_count_ips_raw.info()

For this one, rows with ASN missing data (NaN) will be deleted, and also with IP range missing data

In [ ]:
df_asn_cat_count_ips_clean = df_asn_cat_count_ips_raw.dropna(subset=['start_ip', 'end_ip', 'asn'])


In [ ]:

df_asn_cat_count_ips_clean.info()
unique_ansns_df_asn_cat_count_ips_clean = df_asn_cat_count_ips_clean.asn.unique()
print(unique_ansns_df_asn_cat_count_ips_clean)
print(len(unique_ansns_df_asn_cat_count_ips_clean))

df_asn_cat_count_ips_clean.rename(columns = {'asn':'ASN'}, inplace = True, errors='ignore')
df_asn_cat_count_ips_clean.rename(columns = {'country':'country2'}, inplace = True, errors='ignore')
df_asn_cat_count_ips_clean.info()
df_asn_cat_count_ips_clean.head()
# TODO: Data formatting

## ASN and categories

In [ ]:
df_cat_as_raw.head()

For this one, all rows must be kept, and summary type function is to be run to inspect possible labels for each category/subcategory level.

In [ ]:
# Find if any row without ASN
df_cat_as_raw_clean = df_cat_as_raw[~df_cat_as_raw['ASN'].isna()]
df_cat_as_raw_clean.info()



unique_ansns_df_cat_as_raw_clean = df_cat_as_raw_clean.ASN.unique()
print(unique_ansns_df_cat_as_raw_clean)
print(len(unique_ansns_df_cat_as_raw_clean))


In [ ]:
df_cat_as_raw_clean.head()

## ASN, org.names and country

In [ ]:
## code for reading JSON files



df_as_info_raw.head()              

For this one, rows with ASN missing data (NaN) will be deleted

In [ ]:
df_as_info_raw_clean = df_as_info_raw[~df_as_info_raw['asn'].isna()]
df_as_info_raw_clean.info()

# TODO: Data fomatting. For instance, asn to number or string (?)

unique_ansns_df_as_info_raw_clean = df_as_info_raw_clean.asn.unique()
print(unique_ansns_df_as_info_raw_clean)
print(len(unique_ansns_df_as_info_raw_clean))



In [ ]:
df_as_info_raw_clean.head()


In [ ]:
# Create new column with ASN as ASXXXX where XXXX is a number

asn_without_format = df_as_info_raw_clean['asn'].values.tolist()
#print(asn_without_format)


asn_with_format = []
for i in range(len(asn_without_format)):
    intValue = int(asn_without_format[i])
    asn_with_format.append('AS' + str(intValue))

print(asn_with_format[-5:])
print(len(asn_with_format))

df_as_info_raw_clean.insert(7, "ASN", asn_with_format)

#df.insert(1, "newcol", [99, 99])


In [ ]:
df_as_info_raw_clean = df_as_info_raw_clean.drop(columns=['asn'])
df_as_info_raw_clean.head()

## Merge two datasets without categories, to get country names, organization names and IP ranges for each ASN)

In [ ]:
# df_asn_cat_count_ips_clean (start_ip	end_ip	country	country_name	continent	continent_name	ASN	as_name	as_domain)
# df_cat_as_raw_clean (ASN	Category 1 - Layer 1	Category 1 - Layer 2	Category 2 - Layer 1	Category 2 - Layer 2)
# df_as_info_raw_clean (changed	country	name	organizationId	source	type	asn	ASN	opaqueId)

df1 = df_asn_cat_count_ips_clean
df2 = df_as_info_raw_clean
df3 = df_cat_as_raw_clean

In [ ]:
#df_merged_without_categories = df_as_info_raw_clean.merge(df_asn_cat_count_ips_clean,how = "outer", on=['ASN'])
asn1 = df1['ASN'].unique()
asn2 = df2['ASN'].unique()
print("Cantidad de ASNs en df1:", len(asn1))
print("Cantidad de ASNs en df2:", len(asn2))

# LEFT merge needs to be used in order to keep rows from df1, which are the ones with ip ranges, which is essential information for next steps
df12 = pd.merge(df1, df2,how = "left", on=['ASN'])
#df12 = df12.drop(columns=['asn'])

unique_ansns_df_merged_without_categories = df12.ASN.unique()

print("Cantidad de ASNs en merged:", len(unique_ansns_df_merged_without_categories))
df12.info()
df12.head()

##### 

## Analyzing distribution of categories dataset

In [ ]:
# TODO: run summary for each col, find most frequent, and up to which layer/level NaNs become vast majority.
df_cat_as_raw_clean.head()

Find out how NaN-populated all columns are

In [ ]:
# Suponiendo que ya tienes un DataFrame df
proporcion_nan = df_cat_as_raw_clean.isna().mean()

# Mostrar la proporción de NaN por cada columna
print(proporcion_nan[0:20])

In [ ]:
import matplotlib.pyplot as plt

prop_nan = proporcion_nan[1:10]*100


# Graficar el histograma
prop_nan.plot(kind='bar', color='skyblue')

# Personalizar el gráfico
plt.title('Proporción de valores NaN')
plt.ylabel('Proporción NaN [%]')
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 100)

# Mostrar el gráfico
plt.show()

In [ ]:
frecuencias = df_cat_as_raw_clean['Category 1 - Layer 1'].value_counts()

import matplotlib.pyplot as plt

# Mostrar las frecuencias
print(frecuencias[0:10])
print(len(frecuencias))

frec = frecuencias[0:10]

# Graficar el histograma
frec.plot(kind='bar', color='skyblue')

# Personalizar el gráfico
plt.title('Frecuencia de los valores en la columna')
plt.xlabel('Valores')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45, ha='right')

# Mostrar el gráfico
plt.show()

In [ ]:
frecuencias = df_cat_as_raw_clean['Category 1 - Layer 2'].value_counts()

import matplotlib.pyplot as plt

# Mostrar las frecuencias
print(frecuencias[0:10])
print(len(frecuencias))

frec = frecuencias[0:20]

# Graficar el histograma
plt.figure(figsize=(20, 6))  # Cambia el tamaño según prefieras (10 pulgadas de ancho, 6 de alto)
frec.plot(kind='bar', color='skyblue')

# Personalizar el gráfico
plt.title('Frecuencia de los valores en la columna')
plt.xlabel('Valores')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45, ha='right')

# Mostrar el gráfico
plt.show()

Now 4 dataframes will be created with selected categories for critical infrastructure, those being:
* Energy (Electric Power Generation, Transmision, Distribution)
* Transportation (Automotive, Air Travel, Trucking, Air ~, Railroad ~)
* Natural Gas Distribution
* Water (~ Supply and Irrigation, ~ Transportation, ~ Travel)


In [ ]:

cat_energ = df_cat_as_raw_clean[(df_cat_as_raw_clean['Category 1 - Layer 2'] == 'Electric Power Generation, Transmission, Distribution')]

cat_trans = df_cat_as_raw_clean[(df_cat_as_raw_clean['Category 1 - Layer 2'] == 'Automotive and Transportation') | 
                                (df_cat_as_raw_clean['Category 1 - Layer 2'] == 'Air Travel') | 
                                (df_cat_as_raw_clean['Category 1 - Layer 2'] == 'Trucking') | 
                                (df_cat_as_raw_clean['Category 1 - Layer 2'] == 'Air Transportation') | 
                                (df_cat_as_raw_clean['Category 1 - Layer 2'] == 'Railroad Transportation')]

cat_natgas = df_cat_as_raw_clean[(df_cat_as_raw_clean['Category 1 - Layer 2'] == 'Natural Gas Distribution')]

cat_water = df_cat_as_raw_clean[(df_cat_as_raw_clean['Category 1 - Layer 2'] == 'Water Supply and Irrigation') | 
                                (df_cat_as_raw_clean['Category 1 - Layer 2'] == 'Water Transportation') | 
                                (df_cat_as_raw_clean['Category 1 - Layer 2'] == 'Water Travel')]
 
cat_civil_eng = df_cat_as_raw_clean[(df_cat_as_raw_clean['Category 1 - Layer 2'] == 'Civil Engineering Construction (Utility Lines, Roads and Bridges)')]

In [ ]:
cat_energ.head()

In [ ]:
cat_trans.head()

In [ ]:
cat_natgas.head()

In [ ]:
cat_water.head()

In [ ]:
cat_civil_eng.head()

In [ ]:
df121 = df12[df12['ASN'].isin(cat_energ.ASN.unique())]
df122 = df12[df12['ASN'].isin(cat_trans.ASN.unique())]
df123 = df12[df12['ASN'].isin(cat_natgas.ASN.unique())]
df124 = df12[df12['ASN'].isin(cat_water.ASN.unique())]
df125 = df12[df12['ASN'].isin(cat_civil_eng.ASN.unique())]

df125_3countries = df125[(df125['country_name'] == 'Spain') | (df125['country_name'] == 'France') |  (df125['country_name'] == 'Italy')]

"""
df121.to_csv('outdf121.csv', index=False)  
df122.to_csv('outdf122.csv', index=False)  
df123.to_csv('outdf123.csv', index=False)  
df124.to_csv('outdf124.csv', index=False)
df125.to_csv('outdf125.csv', index=False)
"""

In [ ]:
df125_3countries.info()
df125_3countries.head()

# Building IP lists

In [ ]:
"""
import ipaddress

# Función para generar el rango de IPs entre dos direcciones IP (IPv4 o IPv6)
def get_ip_range(start_ip, end_ip):
    start = ipaddress.ip_address(start_ip)
    end = ipaddress.ip_address(end_ip)
    
    # Asegurarse de que ambas direcciones sean del mismo tipo (IPv4 o IPv6)
    if start.version != end.version:
        raise ValueError(f"Las direcciones IP {start_ip} y {end_ip} no son del mismo tipo (IPv4 vs IPv6).")
    
    return [str(ip) for ip in range(int(start), int(end) + 1)]

# Lista para almacenar todas las direcciones IP
all_ips = []

# Iterar sobre cada fila del DataFrame
for index, row in df121.iterrows():
    ip_range = get_ip_range(row['start_ip'], row['end_ip'])
    all_ips.extend(ip_range)

# Imprimir la lista final de IPs
print(all_ips[0:10])

# Guardar todas las IPs en un archivo de texto
with open("ips_output.txt", "w") as file:
    for ip in all_ips:
        file.write(ip + "\n")

# Imprimir la lista final de IPs
print(f"Las direcciones IP se han guardado en 'ips_output.txt'.")
"""

# TARGET

I will try to get a table with the following columns:
* Org. name
* First/last IP in org IP range
* Org. ASN
* Org. Country

Then I will filter and create a baseline org. profile (and methodology to get all data from orgs which match the baseline profile)

Final product will be raw list of IPs to run on masscan/EyeWitness

## B class IP addresses

In [ ]:
import ipaddress


# Función para verificar si una IP está en el rango de clase B y es IPv4
def is_class_b_range(start_ip, end_ip):
    try:
        # Convertir a objetos IPv4 e ignorar si es IPv6
        start_ip = ipaddress.ip_address(start_ip)
        end_ip = ipaddress.ip_address(end_ip)
        
        if not isinstance(start_ip, ipaddress.IPv4Address) or not isinstance(end_ip, ipaddress.IPv4Address):
            return False
    
        class_b_start = ipaddress.IPv4Address('128.0.0.0')
        class_b_end = ipaddress.IPv4Address('191.255.255.255')

        return (class_b_start <= start_ip <= class_b_end) and (class_b_start <= end_ip <= class_b_end)

    except ValueError:
    # Si la IP no es válida, retornar False
        return False

    
# Filtrar las filas que cumplen con el criterio de clase B
df12_europe = df12[(df12['continent_name'] == 'Europe')]
df12_asia = df12[(df12['continent_name'] == 'Asia')]
asia_class_b = df12_asia[df12_asia.apply(lambda row: is_class_b_range(row['start_ip'], row['end_ip']), axis=1)]

europe_class_b = df12_europe[df12_europe.apply(lambda row: is_class_b_range(row['start_ip'], row['end_ip']), axis=1)]

# Mostrar el resultado
europe_class_b.info()
europe_class_b.head()

In [ ]:
# europe_class_b.to_csv('europe_class_b.csv', index=False)

In [ ]:
# Mostrar el resultado
asia_class_b.info()
asia_class_b.head()

In [ ]:
energ_asia_class_b = asia_class_b[asia_class_b['ASN'].isin(cat_energ.ASN.unique())]
trans_asia_class_b = asia_class_b[asia_class_b['ASN'].isin(cat_trans.ASN.unique())]
natgas_asia_class_b = asia_class_b[asia_class_b['ASN'].isin(cat_natgas.ASN.unique())]
water_asia_class_b = asia_class_b[asia_class_b['ASN'].isin(cat_water.ASN.unique())]


#asia_class_b.to_csv('asia_class_b.csv', index=False)

In [ ]:
energ_asia_class_b.info()
energ_asia_class_b.head()

In [ ]:
trans_asia_class_b.info()
trans_asia_class_b.head()

In [ ]:
natgas_asia_class_b.info()
natgas_asia_class_b.head()

In [ ]:
water_asia_class_b.info()
water_asia_class_b.head()

# ASIA CRIT INF

In [ ]:
asia_crit_inf_class_B = pd.concat([energ_asia_class_b, trans_asia_class_b, natgas_asia_class_b, water_asia_class_b], axis=0)
#asia_crit_inf.to_csv('asia_crit_inf.csv', index=False)

In [ ]:
energ_asia = df12_asia[df12_asia['ASN'].isin(cat_energ.ASN.unique())]
trans_asia = df12_asia[df12_asia['ASN'].isin(cat_trans.ASN.unique())]
natgas_asia = df12_asia[df12_asia['ASN'].isin(cat_natgas.ASN.unique())]
water_asia = df12_asia[df12_asia['ASN'].isin(cat_water.ASN.unique())]

energ_asia.to_csv('energ_asia.csv', index=False)
trans_asia.to_csv('trans_asia.csv', index=False)
natgas_asia.to_csv('natgas_asia.csv', index=False)
water_asia.to_csv('water_asia.csv', index=False)

asia_crit_inf = pd.concat([energ_asia, trans_asia, natgas_asia, water_asia], axis=0)
asia_crit_inf.info()
asia_crit_inf.head()

In [ ]:
# asia_crit_inf.to_csv('asia_crit_inf.csv', index=False)

import ipaddress

def get_ip_version(start_ip):
    ip_obj = ipaddress.ip_address(start_ip)
    return 'IPv4' if ip_obj.version == 4 else 'IPv6'

# Función para generar el rango de IPs entre dos direcciones IP (IPv4 o IPv6)
def count_ip_range(start_ip, end_ip):
    start = ipaddress.ip_address(start_ip)
    end = ipaddress.ip_address(end_ip)
    
    # Asegurarse de que ambas direcciones sean del mismo tipo (IPv4 o IPv6)
    if start.version != end.version:
        raise ValueError(f"Las direcciones IP {start_ip} y {end_ip} no son del mismo tipo (IPv4 vs IPv6).")
    
    # This next line is for IP list generation, for the moment it is disabled
    # return [str(ipaddress.ip_address(ip)) for ip in range(int(start), int(end) + 1)]
    return int(end) - int(start) + 1

In [ ]:
df12['ip_version'] = df12.apply(lambda row: get_ip_version(row['start_ip']), axis=1)


df12_IPv4 = df12[(df12['ip_version'] == 'IPv4')]
df12_europe = df12_IPv4[(df12_IPv4['continent_name'] == 'Europe')]
df12_asia = df12_IPv4[(df12_IPv4['continent_name'] == 'Asia')]

civil_eng_europe = df12_europe[df12_europe['ASN'].isin(cat_civil_eng.ASN.unique())]
civil_eng_asia = df12_asia[df12_asia['ASN'].isin(cat_civil_eng.ASN.unique())]


civil_eng_europe['ip_count'] = civil_eng_europe.apply(lambda row: count_ip_range(row['start_ip'], row['end_ip']), axis=1)
civil_eng_asia['ip_count'] = civil_eng_asia.apply(lambda row: count_ip_range(row['start_ip'], row['end_ip']), axis=1)


civil_eng_europe.info()
civil_eng_europe.head()


In [ ]:
# unique countries
print(len(civil_eng_europe.country_name.unique()))

# unique org
print(len(civil_eng_europe.as_name.unique()))

# amount of IPs
tot_ips = 0
ip_counts = civil_eng_europe['ip_count']
for i in range(len(civil_eng_europe)):
    tot_ips = tot_ips + ip_counts.iloc[i]

print(tot_ips)

In [ ]:
civil_eng_asia.info()
civil_eng_asia.head()

In [ ]:
# unique countries
print(len(civil_eng_asia.country_name.unique()))

# unique org
print(len(civil_eng_asia.as_name.unique()))

# amount of IPs
tot_ips = 0
ip_counts = civil_eng_asia['ip_count']
for i in range(len(civil_eng_asia)):
    tot_ips = tot_ips + ip_counts.iloc[i]

print(tot_ips)

In [ ]:
civil_eng_asia.to_csv('civil_eng_asia.csv', index=False)